In [ ]:
Redux-Toolkit gồm những gì ở bên trong:
- configureStore(): 
    + Là một hàm wrap (bọc) bên trong dùng `createStore` => Cuối cùng cũng chỉ để tạo ra `store`
    + Tự combine các reducer (rootReducer)
    + Có sẵn redux-thunk  

- Hàm createReducer():
    + Không cần viết câu lệnh switch-case
    + Sử dụng thư viện `immer`: Gán trực tiếp vào một object & tự động luôn tạo object mới (immutable)
        => Luôn luôn cập nhật re-render
        => Viết theo kiểu tự nhiên

- Hàm createAction():
    + Hàm tạo ra action

- Hàm createSlice():
    + Hàm tạo ra các slice con (tương tự như reducer con)
    VD:
    src/
    └── features/
        ├── address/    --> slice
        └── product/    --> slice



In [ ]:
/* Reset dự án
- Để lại utils/httpRequest.js => Cấu hình Axios
- Để lại services => gọi API
*/

In [ ]:
// src/services/product/services.js

import httpRequest from "@/utils/httpRequest";

export async function getProducts() {
    const response = await httpRequest.get("/products");
    return response;
}

In [ ]:
// src/services/product/index.js

export * from './services'

In [ ]:
// src/utils/httpRequest.js

import axios from "axios";

const httpRequest = axios.create({
    baseURL: "https://api01.f8team.dev/api",
});

httpRequest.interceptors.response.use((response) => {
    return response.data;
});

export default httpRequest;


BẮT ĐẦU REDUX-TOOLKIT

In [ ]:
npm install @reduxjs/toolkit react-redux

In [ ]:
// Tạo file: src/store.js

import { configureStore } from "@reduxjs/toolkit";

const store = configureStore({
    reducer: {}, // Bản chất của reducer: Viết cái gì vào bên trong {...} --> sẽ được truyền vào combine reducer
});

export { store };

In [ ]:
// src/main.jsx

//...
// Cấu hình Provider và truyền props store
import { Provider as ReduxProvider } from "react-redux";
import { store } from "./store";
createRoot(document.getElementById("root")).render(
    <ReduxProvider store={store}>
        <App />
    </ReduxProvider>,
);


In [ ]:
// Tạo file: src/features/product/productSlice.js

// 1. Import createSlice
import { createSlice } from "@reduxjs/toolkit";

// 2. Khởi tạo state con của slice
const initialState = {
    products: [],
    value: 0,
};

// 3. Khởi tạo slice
const productSlice = createSlice({
    name: "product", // NAMESPACE
    initialState, // Giá trị ban đầu của state
    reducers: {
        // Các logic thực hiện hành động
        increase(state) {
            state.value++;
        },
    },
});
/* 
Hàm createSlice sẽ trả về một object:
- Object có trường thuộc tính là actions và reducer
- actions là một object chứa các phương thức (action) mà chúng ta đã cấu hình như increase... (Vì mỗi phương thức truyền vào reducer sẽ tự động sinh ra một cái action tương ứng)
- các phương thức increase... chính là một hàm action creator. Khi gọi hàm sẽ trả ra object có type, payload
*/
export const { increase } = productSlice.actions; 
// increase chính là một hàm, khi gọi hàm () sẽ trả ra một object action có { type: NAMESPACE/tên_phương_thức }
// VD: {type: product/increase, payload:...}

/* productSlice cuối cùng sẽ cắm vào combine reducer */
export default productSlice.reducer;


In [ ]:
// Tạo file: src/store.js

//...

// 1. import reducer
import productReducer from "./features/product/productSlice";

const store = configureStore({
    reducer: {
        product: productReducer // 2. Thêm reducer con
    },
});

export { store };

In [ ]:
// src/pages/Home/index.jsx

import { increase } from "@/features/product/productSlice";
import React from "react";
import { useDispatch, useSelector } from "react-redux";

function Home() {
    // 1. Lấy ra biến counter từ giá trị value của product
    const counter = useSelector((state) => state.product.value);
    const dispatch = useDispatch();

    // increase() chính là một hàm trả ra object có type, payload
    return <h1 onClick={() => dispatch(increase())}>Count is {counter}</h1>;
}

export default Home;


2. `createAsyncThunk`
=> Giúp tạo ra một actions có thể gọi được API, bất đồng bộ

In [ ]:
/* 
Thunk là gì?
- Thunk là một hàm bao bọc logic (function) mà chưa thực hiện ngay, mà sẽ được thực hiện sau.
- Trong Redux: Thunk cho phép viết action creator trả về một function thay vì object.
    + Function này sẽ nhận dispatch (và getState) để bạn thực hiện side effect, ví dụ gọi API, setTimeout, v.v.
*/
// Ví dụ
const incrementAsync = (amount) => {
  return (dispatch, getState) => {
    setTimeout(() => {
      dispatch({ type: 'counter/increment', payload: amount });
    }, 1000);
  };
};

// incrementAsync là "thunk action creator".
// Nó trả về một function.
// + Function này được middleware Redux Thunk nhận và thực thi.
// + Bên trong function, bạn vẫn có thể dispatch các action object bình thường.


In [ ]:
/* createAsyncThunk là gì?
- Cú pháp tiện lợi để tạo sẵn thunk action creator cho async logic.
- Nó tự động tạo ra 3 action object pending / fulfilled / rejected, bạn không phải viết thủ công.
*/

// Ví dụ:
const fetchUserById = createAsyncThunk(
  'users/fetchByIdStatus', // type
  async (userId, thunkAPI) => {
    const response = await userAPI.fetchById(userId);
    return response.data; // Đây sẽ là payload cho action fulfilled
  }
);

// fetchUserById → là "thunk action creator"
// Khi dispatch(fetchUserById(123)):
// 1. RTK tự dispatch fetchUserById.pending
// 2. Chạy async function
// 3. Nếu thành công, dispatch fetchUserById.fulfilled với payload
// 4. Nếu lỗi, dispatch fetchUserById.rejected

In [ ]:
/* 
Vậy tại sao dispatch lại nhận được function?
- Redux bình thường không nhận function.
- Khi dùng middleware Redux Thunk, middleware này sẽ kiểm tra xem action là function hay object:
    + Nếu là object → cho qua, bình thường.
    + Nếu là function → gọi function đó, truyền dispatch và getState vào.
=> Vì vậy, thunk action creator chính là cầu nối, tạo ra function để middleware xử lý async logic.
*/

In [ ]:
// src/features/product/productSlice

// 1. Lấy hàm getProducts từ service
import { getProducts } from "@/services/product";

// 2. import createAsyncThunk
import { createAsyncThunk, createSlice } from "@reduxjs/toolkit";

//...

/* 3. Tạo một thunk action creator */
const fetchProducts = createAsyncThunk(
    "product/fetchProducts", // type
    async () => {
        const response = await getProducts();
        return response.data.items; // payload
    },
);

const productSlice = createSlice({
    name: "product",
    initialState,
    reducers: {
        //...
    },

    /* 4. reducer xử lý thao tác bất đồng bộ */
    extraReducers: (builder) => {
        // Khi ở trạng thái pending...

        /* fulfilled */
        builder.addCase(fetchProducts.fulfilled, (state, action) => {
            state.products = action.payload; // action.payload: chính là response.data.items;
        });

        // Khi ở trạng thái reject...
    },
});


export { fetchProducts } 

In [ ]:
//src/pages/Home/index.jsx

import { fetchProducts } from "@/features/product/productSlice";
import React, { useEffect } from "react";
import { useDispatch, useSelector } from "react-redux";

function Home() {
    /* Custom Hook START */
    const dispatch = useDispatch();
    const products = useSelector((state) => state.product.products);

    useEffect(() => {
        dispatch(fetchProducts());
    }, [dispatch]);
    /* Custom Hook END */

    return (
        <div>
            <h1>Product List</h1>
            <ul>
                {products.map((product) => (
                    <li key={product.id}>
                        {product.id}.{product.title}
                    </li>
                ))}
            </ul>
        </div>
    );
}

export default Home;
